In [445]:
c_ = 0.1 * tf.Variable(tf.truncated_normal([4,4], dtype=tf.float32))
h_ = 0.1 * tf.Variable(tf.truncated_normal([4,4], dtype=tf.float32))

In [494]:
import numpy as np
import tensorflow as tf
sess = tf.Session()


c_ = 0.1 * tf.constant([[0,0,0,0], # 4*4
                        [0,0,0,0],
                        [0,0,0,0],
                        [0,0,0,0]], dtype=tf.float32)

h_ = 0.1 * tf.constant([[0,0,0,0], # 4*4
                       [0,0,0,0],
                       [0,0,0,0],
                       [0,0,0,0]], dtype=tf.float32)


#he
#el
#ll
#lo

seq_len = len(sess.run(x))#cell 갯수, 인풋이 몇 덩어리인지
#num_units = len(sess.run(c_))# hiddenlayer 갯수, c값이 몇 덩어리인지

#seq_len = len(sess.run(x))#cell 갯수, 인풋이 몇 덩어리인지
#num_units = len(sess.run(c_))# hiddenlayer 갯수, c값이 몇 덩어리인지
#x행 = h행
#c행 = 노상관
#h행 = c행
##########달라도 되는건 x열뿐############
class lstm:
    def build(c, h):
        args = tf.concat((x,h), axis=1)
        print(args)

        out_size = 4 * num_units
        proj_size = args.shape[-1]
        print(out_size)
        print(proj_size)

        weights = tf.ones([proj_size, out_size]) * 0.5
        print(weights)


        out = tf.matmul(args, weights)
        print(out)

        bias = tf.ones([out_size]) * 0.5
        print(bias)

        concat = out + bias
        print(concat)

        i, j, f, o = tf.split(concat, 4, 1)
        print(i)
        print(j)
        print(f)
        print(o)

        g = tf.tanh(j)
        print(g)

        def sigmoid_array(x):
            return 1 / (1 + tf.exp(-x))

        forget_bias = 1.0

        sigmoid_f = sigmoid_array(f + forget_bias)
        print(sigmoid_f)

        sigmoid_array(i) * g

        new_c = c * sigmoid_f + sigmoid_array(i) * g
        print(new_c)

        new_h = tf.tanh(new_c) * sigmoid_array(o)
        print(new_h)

        print('\n new_h:',new_h)
        print('\n new_c',new_c)

#        print(res[1].h)
#        print(res[1].c)

        return new_c, new_h

In [495]:
#ta = tf.TensorArray(size=1, dtype=tf.float32)
ta_c = tf.TensorArray(size=seq_len, dtype=tf.float32)
ta_h = tf.TensorArray(size=seq_len, dtype=tf.float32)

In [530]:
def body(last_state, last_output, step):
    
    output = lstm.build(last_state, last_output)[0]
    state = lstm.build(last_state, last_output)[1]


    return state, output, tf.add(step, 1)
    

timesteps = seq_len

steps = lambda a, b, step: tf.less(step, timesteps)

lstm_output, lstm_state, step = tf.while_loop(steps, body, (c_, h_, 0), parallel_iterations=20)


Tensor("while_211/concat:0", shape=(4, 8), dtype=float32)
16
8
Tensor("while_211/mul:0", shape=(8, 16), dtype=float32)
Tensor("while_211/MatMul:0", shape=(4, 16), dtype=float32)
Tensor("while_211/mul_1:0", shape=(16,), dtype=float32)
Tensor("while_211/add:0", shape=(4, 16), dtype=float32)
Tensor("while_211/split:0", shape=(4, 4), dtype=float32)
Tensor("while_211/split:1", shape=(4, 4), dtype=float32)
Tensor("while_211/split:2", shape=(4, 4), dtype=float32)
Tensor("while_211/split:3", shape=(4, 4), dtype=float32)
Tensor("while_211/Tanh:0", shape=(4, 4), dtype=float32)
Tensor("while_211/truediv:0", shape=(4, 4), dtype=float32)
Tensor("while_211/add_5:0", shape=(4, 4), dtype=float32)
Tensor("while_211/mul_5:0", shape=(4, 4), dtype=float32)

 new_h: Tensor("while_211/mul_5:0", shape=(4, 4), dtype=float32)

 new_c Tensor("while_211/add_5:0", shape=(4, 4), dtype=float32)
Tensor("while_211/concat_1:0", shape=(4, 8), dtype=float32)
16
8
Tensor("while_211/mul_6:0", shape=(8, 16), dtype=float32)

In [537]:
xdata = [[1,0,0,0],# 4*4  #h
        [0,1,0,0], #e
        [0,0,1,0], #l
        [0,0,0,1]] #o

ydata = [[0,1,0,0], # 5*4
        [0,0,1,0],
        [0,0,1,0],
        [0,0,0,1]]

X = tf.placeholder(dtype=tf.float32, shape=[None, 4])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 4])

W = tf.Variable(tf.random_normal([4, 1]))
b = tf.Variable(tf.random_normal([1]))

logits = tf.matmul(sess.run(ta_h.read(seq_len-1)), W) + b

In [538]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=Y))
train = tf.train.AdamOptimizer(0.0001).minimize(cost)

In [539]:
is_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

In [540]:
sess.run(tf.global_variables_initializer())

In [541]:
for i in range(seq_len):
    sess.run(tf.while_loop(steps, body, (c_, h_, 0), parallel_iterations=20), feed_dict={X:xdata, Y:ydata})
    a = sess.run(accuracy,feed_dict={X:xdata, Y:ydata})
    print("##########################",a,"#####################")

Tensor("while_224/concat:0", shape=(4, 8), dtype=float32)
16
8
Tensor("while_224/mul:0", shape=(8, 16), dtype=float32)
Tensor("while_224/MatMul:0", shape=(4, 16), dtype=float32)
Tensor("while_224/mul_1:0", shape=(16,), dtype=float32)
Tensor("while_224/add:0", shape=(4, 16), dtype=float32)
Tensor("while_224/split:0", shape=(4, 4), dtype=float32)
Tensor("while_224/split:1", shape=(4, 4), dtype=float32)
Tensor("while_224/split:2", shape=(4, 4), dtype=float32)
Tensor("while_224/split:3", shape=(4, 4), dtype=float32)
Tensor("while_224/Tanh:0", shape=(4, 4), dtype=float32)
Tensor("while_224/truediv:0", shape=(4, 4), dtype=float32)
Tensor("while_224/add_5:0", shape=(4, 4), dtype=float32)
Tensor("while_224/mul_5:0", shape=(4, 4), dtype=float32)

 new_h: Tensor("while_224/mul_5:0", shape=(4, 4), dtype=float32)

 new_c Tensor("while_224/add_5:0", shape=(4, 4), dtype=float32)
Tensor("while_224/concat_1:0", shape=(4, 8), dtype=float32)
16
8
Tensor("while_224/mul_6:0", shape=(8, 16), dtype=float32)